In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
df_2016 = pd.read_csv('2016_data_integrated.csv', index_col='Datetime', parse_dates=True)
df_2017 = pd.read_csv('2017_data_integrated.csv', index_col='Datetime', parse_dates=True)
df_2018 = pd.read_csv('2018_data_integrated.csv', index_col='Datetime', parse_dates=True)
df_2019 = pd.read_csv('2019_data_integrated.csv', index_col='Datetime', parse_dates=True)
df_2020 = pd.read_csv('2020_data_integrated.csv', index_col='Datetime', parse_dates=True)
df_2021 = pd.read_csv('2021_data_integrated.csv', index_col='Datetime', parse_dates=True)
df_2022 = pd.read_csv('2022_data_integrated.csv', index_col='Datetime', parse_dates=True)
df_2020_negative = pd.read_csv('df_Bonn2020.csv', index_col='Time', parse_dates=True)
df_2021_negative = pd.read_csv('df_Bonn2021.csv', index_col='Time', parse_dates=True)
df_2022_negative = pd.read_csv('df_Bonn2022.csv', index_col='Datum/Uhrzeit', parse_dates=True)

In [3]:
df_till2019 = pd.concat([df_2016, df_2017, df_2018, df_2019])
df_from2020 = pd.concat([df_2020_negative, df_2021_negative, df_2022_negative])
df_from2020_weather = pd.concat([df_2020, df_2021, df_2022])
df_from2020_weather = df_from2020_weather['Temperature (°C)', 'Precipitations (mm)', 'Wind Speed (m/s)']
df_from2020 = pd.concat([df_from2020_weather, df_from2020], axis=1)

KeyError: ('Temperature (°C)', 'Precipitations (mm)', 'Wind Speed (m/s)')

In [ ]:
new_column_names = {
    'Kennedybrücke': 'KennedyBrücke',
    'Nordbrücke': 'NordBrücke',
    'Südbrücke': 'SüdBrücke',
    'Von-Sandt-Ufer': 'VonSandtUfer',
    'Brühler Straße': 'BrühlerStraße',
    'Wilhelm-Spiritus-Ufer': 'WilhelmSpiritusUfer',
    'Mc Cloy Weg': 'McCloyWeg',
    'Weg auf Damm Neil': 'WegDammBonnBeuel'
}
df_from2020 = df_from2020.rename(columns=new_column_names)
df_from2020.columns

In [ ]:
df_till2019.to_csv('df_till2019.csv')
df_from2020.to_csv('df_from2020.csv')

In [ ]:
df_all = pd.concat([df_till2019, df_from2020])
df_all

In [ ]:
df_standort = df_all[['KennedyBrücke', 'NordBrücke', 'SüdBrücke', 'Estermannufer',
       'VonSandtUfer', 'Rhenusallee', 'BrühlerStraße',
       'WilhelmSpiritusUfer', 'McCloyWeg', 'WegDammBonnBeuel']]

In [ ]:
plt.figure(figsize=(20,6))
df_standort.plot()
plt.title('2016-2022')
plt.ylabel('Anzahl Fahrräder')
plt.xlabel('Jahr')
plt.grid()
plt.show()

### 1. Ansatz: Lücken füllen mit berechneten monatlichen Mittelwerten

In [ ]:
df_all[df_all < 0] = np.nan

In [ ]:
#Mittelwerte aus den Monaten
kennedybrücke_monthly_median = df_all.groupby(df_all.index.month)['Kennedybrücke'].median()
nordbrücke_monthly_median = df_all.groupby(df_all.index.month)['Nordbrücke'].median()
südbrücke_monthly_median = df_all.groupby(df_all.index.month)['Südbrücke'].median()
estermannufer_monthly_median = df_all.groupby(df_all.index.month)['Estermannufer'].median()
von_sandt_ufer__monthly_median = df_all.groupby(df_all.index.month)['Von-Sandt-Ufer'].median()
rhenusallee_monthly_median = df_all.groupby(df_all.index.month)['Rhenusallee'].median()
brühler_straße_monthly_median = df_all.groupby(df_all.index.month)['Brühler Straße'].median()
wilhelm_spiritus_ufer_monthly_median = df_all.groupby(df_all.index.month)['Wilhelm-Spiritus-Ufer'].median()
mccloy_weg_monthly_median = df_all.groupby(df_all.index.month)['Mc Cloy Weg'].median()
weg_auf_damm_neil__monthly_median = df_all.groupby(df_all.index.month)['Weg auf Damm Neil'].median()

df_medians = pd.concat([kennedybrücke_monthly_median, nordbrücke_monthly_median, südbrücke_monthly_median, 
                       estermannufer_monthly_median, von_sandt_ufer__monthly_median, rhenusallee_monthly_median,
                       brühler_straße_monthly_median, wilhelm_spiritus_ufer_monthly_median, mccloy_weg_monthly_median,
                       weg_auf_damm_neil__monthly_median], axis=1)
df_medians = df_medians.astype(int)
df_medians

In [ ]:
missing_values = df_all.isna()

#Imputieren
df_monthly_avg_imputed = df_all.copy()

#Über alle Standorte iterieren
for standort in df_monthly_avg_imputed.columns:
    
    monthly_medians = df_medians[standort]
    #Über alle Monate iterieren
    for month in range(1, 13):
       
        df_monthly_avg_imputed.loc[df_monthly_avg_imputed.index.month == month, standort] = np.where(
            #Bedingung: Wert ist als 'missing markiert'
            missing_values.loc[df_monthly_avg_imputed.index.month == month, standort],
            #True: imputiere Mittelwert
            monthly_medians[month],
            #Sonst: nehme Originalmesswert
            df_monthly_avg_imputed.loc[df_monthly_avg_imputed.index.month == month, standort])

In [ ]:
df_all.loc['2017-03-26 02:00:00']

In [ ]:
df_monthly_avg_imputed.loc['2017-03-26 02:00:00']

### 2. Ansatz: Seasonal Decomposition mit statsmodels

In [ ]:
df_stl_imputed = df_all.copy()

def stl_berechnen(df, standort):
    '''
    Ersetzt die fehlenden Werte eines Standorts durch Saison-Trend-Zerlegung
    
    Parameter: df: der Datensatz
               standort: der Standort/ die Spalte
               
    Returns:   berechnete_werte: die komplette Zeitreihe(Standort) mit gefüllten Lücken
    
    '''
    stl = STL(df[standort], period=24, seasonal=13)
    result = stl.fit()
    berechnete_werte = result.trend + result.seasonal + result.resid
    return berechnete_werte

for standort in df_stl_imputed.columns:
    
    stl_values = stl_berechnen(df_stl_imputed, standort)
    df_stl_imputed[standort] = np.where(df_stl_imputed[standort], stl_values, df_stl_imputed[standort])

In [ ]:
df_stl_imputed.loc['2017-03-26 02:00:00']

### 3. Ansatz: scikit Methoden 

In [ ]:
#Fehlende Werte durch häufigst vorkommenden Wert ersetzen
imputer = SimpleImputer(strategy='most_frequent')
df_most_frequent_imputed = pd.DataFrame(imputer.fit_transform(df_all), columns=df_all.columns, index=df_all.index)
df_most_frequent_imputed.loc['2017-03-26 02:00:00']

In [ ]:
imputer2 = KNNImputer(n_neighbors=2)
df_kNN_imputed = pd.DataFrame(imputer2.fit_transform(df_all), columns=df_all.columns, index=df_all.index)
df_kNN_imputed.loc['2017-03-26 02:00:00']

In [ ]:
df_all.loc['2017-03-26 01:00:00']

In [ ]:
df_all.loc['2017-03-26 03:00:00']

### 4. Ansatz: Gewichtung der monatlichen Mittelwerte durch Uhrzeit

In [ ]:
#Mittelwerte der Stunden
kennedybrücke_hourly_median = df_all.groupby(df_all.index.hour)['Kennedybrücke'].median()
nordbrücke_hourly_median = df_all.groupby(df_all.index.hour)['Nordbrücke'].median()
südbrücke_hourly_median = df_all.groupby(df_all.index.hour)['Südbrücke'].median()
estermannufer_hourly_median = df_all.groupby(df_all.index.hour)['Estermannufer'].median()
von_sandt_ufer_hourly_median = df_all.groupby(df_all.index.hour)['Von-Sandt-Ufer'].median()
rhenusallee_hourly_median = df_all.groupby(df_all.index.hour)['Rhenusallee'].median()
brühler_straße_hourly_median = df_all.groupby(df_all.index.hour)['Brühler Straße'].median()
wilhelm_spiritus_ufer_hourly_median = df_all.groupby(df_all.index.hour)['Wilhelm-Spiritus-Ufer'].median()
mccloy_weg_hourly_median = df_all.groupby(df_all.index.hour)['Mc Cloy Weg'].median()
weg_auf_damm_neil_hourly_median = df_all.groupby(df_all.index.hour)['Weg auf Damm Neil'].median()

df_medians_hourly = pd.concat([kennedybrücke_hourly_median, nordbrücke_hourly_median, südbrücke_hourly_median, 
                       estermannufer_hourly_median, von_sandt_ufer_hourly_median, rhenusallee_hourly_median,
                       brühler_straße_hourly_median, wilhelm_spiritus_ufer_hourly_median, mccloy_weg_hourly_median,
                       weg_auf_damm_neil_hourly_median], axis=1)
df_medians_hourly = df_medians_hourly.astype(int)
df_medians_hourly

In [ ]:
correlation_matrix = df_medians.corrwith(df_medians_hourly, axis=0)
correlation_matrix

In [ ]:
def gewichtete_berechnung(standort, monat_mediane, stunde_mediane, correlation, monat, stunde):
    
    return correlation[standort] * monat_mediane.loc[monat] + (1-correlation[standort]) * stunde_mediane[stunde]


df_weighted_imputed = df_all.copy()

#Über alle Standorte iterieren
for standort in df_weighted_imputed.columns:
    monat_mediane = df_medians[standort]
    stunde_mediane = df_medians_hourly[standort]
    
    #Über alle Monate iterieren
    for month in range(1, 13):
        
        for hour in range(0, 24):
            df_weighted_imputed.loc[
                (df_weighted_imputed.index.month == month) & (df_weighted_imputed.index.hour == hour), standort
            ] = np.where(
                # Bedingung: Wert ist als 'missing markiert'
                missing_values.loc[
                    (df_monthly_avg_imputed.index.month == month) & (df_monthly_avg_imputed.index.hour == hour), standort
                ],
                # True: imputiere gewichteten Mittelwert
                gewichtete_berechnung(standort, monat_mediane, stunde_mediane, correlation_matrix, month, hour),
                # Sonst: nehme Originalmesswert
                df_monthly_avg_imputed.loc[
                    (df_monthly_avg_imputed.index.month == month) & (df_monthly_avg_imputed.index.hour == hour), standort
                ]
            )

In [ ]:
df_weighted_imputed.loc['2017-03-26 02:00:00']

In [ ]:
df_weighted_imputed.loc['2017-03-26 01:00:00']

In [ ]:
df_weighted_imputed.loc['2017-03-26 03:00:00']

### Abschließend: Ausreißer beseitigen

In [ ]:
outliers_mask = df_weighted_imputed > 1400
df_shifted = df_weighted_imputed.shift(12)

df_clean = df_weighted_imputed.where(~outliers_mask, df_shifted)
df_clean = df_clean.round().astype(int)

In [ ]:
plt.figure(figsize=(20,6))
df_clean.plot()
plt.title('2016-2022')
plt.ylabel('Anzahl Fahrräder')
plt.xlabel('Jahr')
plt.grid()
plt.show()

In [ ]:
df_clean.to_csv('df_clean_standorte.csv')